In [1]:
!wget https://raw.githubusercontent.com/bakrianoo/aravec/master/utilities.py
!gdown 1pHt2cR21oQyeFydiYfF3lehKPm6qon5a
!gdown 1dgZvhDEp0xHvu0jCLNNW_s1mAfvnM8jU

--2023-01-07 10:43:17--  https://raw.githubusercontent.com/bakrianoo/aravec/master/utilities.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2412 (2.4K) [text/plain]
Saving to: ‘utilities.py’

utilities.py        100%[===================>]   2.36K  --.-KB/s    in 0s      

2023-01-07 10:43:17 (34.7 MB/s) - ‘utilities.py’ saved [2412/2412]

Downloading...
From: https://drive.google.com/uc?id=1pHt2cR21oQyeFydiYfF3lehKPm6qon5a
To: /content/train_Arabic_tweets_negative_20190413.tsv
100% 2.78M/2.78M [00:00<00:00, 215MB/s]
Downloading...
From: https://drive.google.com/uc?id=1dgZvhDEp0xHvu0jCLNNW_s1mAfvnM8jU
To: /content/train_Arabic_tweets_positive_20190413.tsv
100% 3.12M/3.12M [00:00<00:00, 177MB/s]


In [2]:
import gensim
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from keras.utils import to_categorical 
import collections
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from keras import models
from keras import layers
from nltk import ngrams
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import sys
import argparse
from utilities import * 

In [3]:
# Get the Embeddings from part 1
!gdown 1Tja3iV4VLQqNy3-y7_Xu0y25B5dUkGBm

# Load Embeddings 
vocab = np.load('Part1_Embeddings.npz')

# check existing words (keys)
print(vocab.files)

# get embeddings for some specific word
print(vocab['الله'])

Downloading...
From: https://drive.google.com/uc?id=1Tja3iV4VLQqNy3-y7_Xu0y25B5dUkGBm
To: /content/Part1_Embeddings.npz
100% 9.18M/9.18M [00:00<00:00, 38.5MB/s]
['UNK', 'بسم', 'الله', 'الرحمن', 'الرحيم', 'الحمد', 'لله', 'والصلاه', 'والسلام', 'رسول', 'وعلي', 'اله', 'وصحبه', 'والاه', 'فهذه', 'المجموعه', 'الاولي', 'مقالات', 'لكبار', 'كتاب', 'العربيه', 'العصر', 'الحديث', 'وقبل', 'الكلام', 'المقالات', 'يحسن', 'المقاله', 'نشاتها', 'ومفهومها', 'وموضوعها', 'وانواعها', 'يدور', 'الفلك', 'فالمقاله', 'المقال', 'باب', 'عظيم', 'ابواب', 'العلم', 'وطريق', 'واسع', 'لنشر', 'الفكر', 'والتاثير', 'الناس', 'ولقد', 'عرفت', 'ظهور', 'المطابع', 'وانتشار', 'الصحافه', 'اواخر', 'القرن', 'الثالث', 'الهجري', 'وذلك', 'انشءت', 'صحيفه', 'الوقاءع', 'المصريه', 'بلغت', 'اوجها', 'منتصف', 'الرابع', 'ازدهرت', 'حركتها', 'البلاد', 'وصارت', 'عماد', 'الكتاب', 'والادباء', 'والقالب', 'يصبون', 'افكارهم', 'وينشرونها', 'وليست', 'غريبه', 'الادب', 'العربي', 'القديم', 'تغيرت', 'صيغها', 'وشروطها', 'فعبدالحميد', 'الكاتب', 'يتكلم', 'الصيد'

In [4]:
nltk.download('stopwords')
nltk.download('punkt')
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ♥️'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations
stop_words = list(set(stopwords.words('arabic')))
print(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['أربعاء', 'تسعين', 'ب', 'ض', 'سابع', 'ثمة', 'أصلا', 'هيّا', 'إليكم', 'هذان', 'وهو', 'أربعمئة', 'ين', 'وما', 'خاصة', 'عن', 'ذال', 'حتى', 'كأنما', 'هَذانِ', 'أولاء', 'أفعل به', 'إياكما', 'لست', 'سنتيم', 'هنالك', 'بسّ', 'اثنين', 'مئة', 'ش', 'ؤ', 'أنتم', 'إياي', 'أي', 'كلاهما', 'راح', 'مه', 'تخذ', 'إذن', 'هلم', 'عامة', 'وراءَك', 'نفس', 'فإن', 'ذو', 'قد', 'ثمانين', 'لما', 'إمّا', 'اللائي', 'لا سيما', 'يمين', 'ا', 'إذاً', 'عند', 'ذهب', 'أيّان', 'أكثر', 'أولالك', 'آض', 'م', 'ولكن', 'لو', 'بين', 'منه', 'ليت', 'ألفى', 'تارة', 'يا', 'فإذا', 'طرا', 'لسن', 'عل', 'بئس', 'شيكل', 'فضلا', 'سحقا', 'طاء', 'ة', 'جانفي', 'قاف', 'ذينك', 'إليك', 'سادس', 'ستمائة', 'آب', 'فيما', 'جمعة', 'لكما', 'تحوّل', 'لكي', 'جوان', 'وا', 'أمس', 'ليس', 'هنا', 'فيها', 'يوان', 'هلّا', 'خلا', 'هذا', 'لسنا', 'هيت', 'رجع', 'عاد', 'تفعلون', 'كأي', 'ثمنمئة', 'عَدَسْ', 'أنى', 'ك', 'سين', 'كليكما', 'لكنما', 'إليكما', 'ضاد', 'حيثما', 'آناء', 'ث', 'أول', 'بماذا', 'بيد', 'ريال', 'حادي', 'هاتان', 'ذ', 'نحن', 'سبع', 'كلّما', 'لي', 'ذوات

In [5]:
train_neg = pd.read_csv('train_Arabic_tweets_negative_20190413.tsv', sep='\t',header=None)

In [6]:
train_pos = pd.read_csv('train_Arabic_tweets_positive_20190413.tsv', sep='\t',header=None)

In [7]:
def remove_diacritics(text):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(arabic_diacritics, ' ', str(text))
    return text

In [8]:
def remove_emoji(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [9]:
def clean_text(text):
    text = "".join([word for word in text if word not in string.punctuation])
    text = remove_emoji(text)
    text = remove_diacritics(text)
    text = re.sub(r'\s*[A-Za-z]+\b', ' ' , text)
    text = re.sub('[a-zA-Z0-9_]|#|http\S+', '', text)

    tokens = word_tokenize(text)
    text = ' '.join([word for word in tokens if word not in stop_words])
    return text

In [10]:
train = {}
for ind in train_neg.index:
  if len(train_neg[1][ind].split()) <185:
    train[train_neg[1][ind]] = 0
  
for ind in train_pos.index:
  if len(train_pos[1][ind].split()) <185:
    train[train_pos[1][ind]] = 1
  # print(train_neg[1][ind])

train_processed = {}
for item in train.items():
  train_processed[clean_text(item[0])]=item[1]
  

In [11]:
train_data = pd.DataFrame(train_processed.items(), columns=['sentence', 'result'])
train_data = train_data.sample(frac=1).reset_index(drop=True)


In [12]:
train_x = train_data['sentence']
train_y = train_data['result']

In [13]:
vocab[word_tokenize(train_x[20])[0]]

array([ 3.45818758e-01,  7.47682750e-01,  4.99484271e-01,  5.15889287e-01,
       -9.55197215e-01,  3.70291173e-01,  5.57814181e-01,  7.67851591e-01,
       -4.76993263e-01, -5.18997252e-01,  5.86728394e-01,  9.85945910e-02,
        2.39072263e-01, -5.64162731e-01,  3.14202011e-02, -5.79975903e-01,
        3.06013495e-01, -2.43315548e-01,  8.38704824e-01,  7.76724100e-01,
        2.90222704e-01,  5.81370831e-01, -1.32859545e-02,  1.13163447e+00,
        6.80532992e-01,  4.33275014e-01, -4.73027259e-01, -3.08532089e-01,
        5.19047856e-01,  5.42888343e-01, -1.60864770e-01, -1.82342276e-01,
       -7.71627247e-01, -4.50501591e-01,  4.57023054e-01,  5.66967785e-01,
        4.82133090e-01, -6.36965362e-03,  7.05089927e-01, -1.74659133e-01,
        1.09915889e-03,  3.72444481e-01,  1.07589893e-01, -1.16643822e+00,
       -1.08468182e-01,  2.41436332e-01,  3.41468267e-02,  1.80849671e-01,
        6.47503078e-01,  7.20451772e-01, -2.68345058e-01,  9.83098090e-01,
       -5.89957349e-02,  

In [14]:
train_x[20]

'اللهم أكرمنا بصباح تصفو النفوس وتطمئن القلوب وتشرق الوجوه بنور التقوى يارب يسر لنآ عسير تعظم عل…'

In [15]:
train_x_input = np.zeros((len(train_x),184,100),dtype="float32")
print(train_x_input.shape)
for i in range(len(train_x)):
  index = 0
  for word in train_x[i].split():
    vec = np.zeros((100,),dtype="float32")
    if word in vocab:
      vec+=vocab[word]
    train_x_input[i,index]=vec
  index+=1
  

(28722, 184, 100)


In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(100,input_shape=(184,100),),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [32]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
# optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [33]:
history = model.fit(x=train_x_input,y=train_y,epochs=10,batch_size=8,verbose=1, validation_split=0.20)

Epoch 1/10
2873/2873 [==============================] - 165s 57ms/step - loss: 0.6941 - accuracy: 0.5105 - val_loss: 0.6930 - val_accuracy: 0.5100
Epoch 2/10
2873/2873 [==============================] - 161s 56ms/step - loss: 0.6932 - accuracy: 0.5084 - val_loss: 0.6942 - val_accuracy: 0.5100
Epoch 3/10
2873/2873 [==============================] - 163s 57ms/step - loss: 0.6932 - accuracy: 0.5119 - val_loss: 0.6930 - val_accuracy: 0.5100
Epoch 4/10
2873/2873 [==============================] - 164s 57ms/step - loss: 0.6932 - accuracy: 0.5086 - val_loss: 0.6931 - val_accuracy: 0.5100
Epoch 5/10
2873/2873 [==============================] - 163s 57ms/step - loss: 0.6935 - accuracy: 0.5083 - val_loss: 0.6933 - val_accuracy: 0.4900
Epoch 6/10
2873/2873 [==============================] - 162s 57ms/step - loss: 0.6931 - accuracy: 0.5122 - val_loss: 0.6929 - val_accuracy: 0.5100
Epoch 7/10
2873/2873 [==============================] - 164s 57ms/step - loss: 0.6930 - accuracy: 0.5099 - val_loss: 0

In [34]:
!gdown 1x2bNmaWCI-ji1-JLSI6N22tIINVv5yhD
!gdown 1L6pLW10tTO1ZM7JZU4MroPABZxAPKPh7

Downloading...
From: https://drive.google.com/uc?id=1x2bNmaWCI-ji1-JLSI6N22tIINVv5yhD
To: /content/test_Arabic_tweets_negative_20190413.tsv
100% 696k/696k [00:00<00:00, 159MB/s]
Downloading...
From: https://drive.google.com/uc?id=1L6pLW10tTO1ZM7JZU4MroPABZxAPKPh7
To: /content/test_Arabic_tweets_positive_20190413.tsv
100% 776k/776k [00:00<00:00, 177MB/s]


In [35]:
test_neg = pd.read_csv('test_Arabic_tweets_negative_20190413.tsv', sep='\t',header=None)

In [36]:
test_pos = pd.read_csv('test_Arabic_tweets_positive_20190413.tsv', sep='\t',header=None)

In [37]:
test = {}
for ind in test_neg.index:
  if len(test_neg[1][ind].split()) <185:
    test[test_neg[1][ind]] = 0
  
for ind in test_pos.index:
  if len(test_pos[1][ind].split()) <185:
    test[test_pos[1][ind]] = 1
  # print(train_neg[1][ind])

In [38]:
test_processed = {}
for item in test.items():
  test_processed[clean_text(item[0])]=item[1]
  

In [39]:
test_data = pd.DataFrame(test_processed.items(), columns=['sentence', 'result'])

In [40]:
test_x = test_data['sentence']
test_y = test_data['result']

In [41]:
test_x_input = np.zeros((len(test_x.values),184,100),dtype="float32")
print(test_x_input.shape)
for i in range(len(test_x.values)):
  index = 0
  for word in test_x[i].split():
    vec = np.zeros((100,),dtype="float32")
    if word in vocab:
      vec+=vocab[word]
    test_x_input[i,index]=vec
  index+=1
  

(8672, 184, 100)


In [42]:
score = model.evaluate(test_x_input, test_y, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.6931782364845276
Test accuracy: 0.5046125650405884
